# SQL: Structured Query Language am Beispiel

Liebe Studierende, SQL ist eine Sprache, die Leben muss und die wir uns am besten mit praktischen Beispielen aneignen. Daher wechseln wir in diese Umgebung, um Theorie mit Praxis zu vermischen. Die Idee ist simpel: _Alles was Sie hier sehen ist in Zellen aufgeteilt_. Es gibt Zellen in denen Text steht (wie hier) und es gibt Zellen in denen Code steht. Code Zellen bestehen aus zwei Teilen: Dem ausführbaren Code und der Ergebniszelle darunter. Um SQL Befehle abzusetzen müssen wir in diese Zellen das Keyword `%sql` (eine Zeile SQL ausführen) oder `%%sql` (die gesamte Zelle enthält SQL Befehle, bitte alle ausführen) schreiben. Der Clou ist, dass **Sie** eingreifen können. Sie können jede Zeile ausführen, indem Sie die Zelle auswählen (blauer Balken an der Seite) und auf `▶` im Menü oben drücken. Dann wird das Ergebnis neu berechnet. Viel Spaß dabei!

In [30]:
!pip install ipython-sql


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%load_ext sql

# Verbinde Dich zu einer in - Memory Datenbank
%sql sqlite:///
%sql PRAGMA foreign_keys = ON

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * sqlite:///
Done.


[]

In [32]:
%%sql
DROP TABLE IF EXISTS enthaelt;
DROP TABLE IF EXISTS Artiel;
DROP TABLE IF EXISTS Bestellung;
DROP TABLE IF EXISTS Kunde;

 * sqlite:///
Done.
Done.
Done.
Done.


[]

Der Aufbau der Lerneinheit ist wie folgt:


<a href="#ddl">1. Data Definition Language</a><br/>
<a href="#11">1.1 Relationen erzeugen</a><br/>
<a href="#12">1.2 Datentypen</a><br/>
<a href="#13">1.3 Spaltenbedingungen</a><br/>
<a href="#14">1.4 Tabellenbedingungen</a><br/>
<a href="#15">1.5 Testen von Constraints</a><br/>
<a href="#16">1.6 Views</a><br/>
<a href="#17">1.7 Zugriffsrechte und Vermischtes</a><br/>

# 1. Data Definition Language 
<a name="ddl">
</a>
Dieser Term bezeichnet den Teil von SQL mit dem

- Relationen erzeugt, geändert und gelöscht werden (nicht deren Inhalte)
- Sichten erzeugt und gelöscht
- Integritätschecks, Invarianten, Trigger, Sequenzen definiert
- Zugriffsrechte und Zugriffsschutz implementiert 

werden. Wenn es um das Anlegen, Ändern und Löschen von Inhalten geht, bemühen sie die <b>Data Manupulation Language (DML)</b>. Im Folgenden werden wir erst nur die Teile suchen (`SELECT ... FROM ...`) und einfügen (`INSERT INTO`) der DML verwenden, Details folgen dann im <a href="#DML">Data Manipulation Language Kapitel</a>. 

In der Box genau über dieser haben wir auch bereits den ersten Befehl gesehen. `DROP TABLE [IF EXISTS] <Tabellenname>` löscht eine Tabelle. Der optionale Zusatz `IF EXISTS` führt dazu, dass keine Fehlermeldung ausgegeben wird, falls eine Tabelle nicht exisitert. 


<a name="11">
</a>

## 1.1 Relationen erzeugen:

[![https://www.sqlite.org/images/syntax/create-table-stmt.gif](https://www.sqlite.org/images/syntax/create-table-stmt.gif)](https://sqlite.org/syntax/create-table-stmt.html)

Neue Relationen (Tabellen) werden in Datenbanken mit dem folgenden Befehl erstellt:
```
CREATE TABLE <Tabellenname> (

 { 
   <Attributname> {<Datentyp> | <Domain>} [<Spaltenbedingungen>]
 |
   <Tabellenbedingung> 
 }

)
```

Wir müssen also mindestens den Namen der Tabelle, deren Attribute und den Datentyp der Attribute kennen. Man kann aber auch eingene Domänen (das sind eigene Datentypen, dazu später), Spaltenbedingungen und Tabellenbedingungen in den `CREATE TABLE` Befehl aufnehmen. Nun schauen wir uns aber erst ein Beispiel an:


In [33]:
%%sql
CREATE TABLE Kunde(
    Kundennummer INTEGER PRIMARY KEY,
    Vorname TEXT,
    Nachname TEXT,
    Wohnort TEXT,
    PLZ INTEGER,
    Adresse TEXT
);

 * sqlite:///
Done.


[]

Glückwunsch, damit haben wir unseren ersten SQL Befehl abgesetzt und falls nichts anderes da steht hat die Datenbank Ihren Befehl auch ausgeführt. Was haben wir also getan? Wir haben eine Tabelle mit den Attributen Kundennummer, Vorname, Nachname, Wohnort, PLZ und Adresse angelegt. **Kundennummer** haben wir zum Primärschlüssel gemacht, den Datentyp haben wir als TEXT angegeben, bei numerischen Werten haben wir eine INTEGER, also eine Ganzzahl gewählt. 

#### Übung
Versuchen Sie es doch gleich einmal selber. Legen Sie eine Tabelle `uebung_telefonbuch` an. In der wollen wir die `kundennummmer` als `INTEGER` und die `telefonnummer` als `TEXT` speichern. 

In [34]:
%%sql
-- Sie sind drann
DROP TABLE IF EXISTS uebung_telefonbuch;
CREATE TABLE uebung_telefonbuch(
);

INSERT INTO uebung_telefonbuch(kundennummer, telefonnummer) VALUES(1, '+49071369672'), 
    (2, '+49022750234'),
    (3, '+492038174371'),
    (4, '+49959271473'),
    (5, '+4991508400545');

 * sqlite:///
Done.
(sqlite3.OperationalError) near ")": syntax error
[SQL: CREATE TABLE uebung_telefonbuch(
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Hat alles geklappt? Bravo!

<a name="12">
    &nbsp
</a>

## 1.2 Datentypen

Hier eine Auflistung der gängigsten Datentypen. Beachten Sie, dass Sie immer für Ihr gewähltes DBMS die Dokumentation der Datentypen überprüfen sollten. Es ist also z.B. wichtig zu verstehen, welchen Datentyp Sie für exakte Zeitpunkte wählen sollten. Bei MYSQL könnte das z.B. DATETIME sein, bei PostgreSQL TIMESTAMP. Auch die Zeitzone muss an dieser Stelle beachtet werden, ist diese bereits enthalten, oder ist das ein eigener Datentyp? Wie Konvertiert die Datenbank? Wird ein String "123" zu 123 automatisch konvertiert, oder ergibt das einen Fehler. Hier zeigen Sich große unterschiede zwischen den Implementierungen.

Name | Bedeutung
-----|--------
`INTEGER / INT` | Ganzzahl
`SMALLINT` | Ganzzahl
`NUMERIC(x,y)` | X Stellige Zahl mit Y Nachkommastellen
`FLOAT(x)` | Gleitkommazahl mit x Nachkommastellen
`TEXT` | Zeichenketten
`CHARACTER(n) / CHAR(n)` | Zeichenketten der Länge n
`VARCHAR(n)` | Variable Zeichenketten der Länge N
`DATE` | Ein Datum
`DATETIME, TIMESTAMP` | Zeitpunkte



<a name="13">
    &nbsp
</a>

## 1.3 Spaltenbedingungen ( Attributbezogene Constraints )

[![https://www.sqlite.org/images/syntax/column-def.gif](https://www.sqlite.org/images/syntax/column-def.gif)](https://sqlite.org/syntax/create-table-stmt.html)

[![https://sqlite.org/images/syntax/column-constraint.gif](https://sqlite.org/images/syntax/column-constraint.gif)](https://sqlite.org/syntax/column-constraint.html)

Um mehr konrolle über die einzelnen Attribute zu erhalten und z.B. Schlüssel zu definieren oder Integritätsbedingunen zu formulieren können wir nach dem Namen des Attributs und dessen Datentyp noch eine oder mehrere Bedingung definieren. Diese beziehen sich eben auf eine Spalte *(auch bei CHECK Bedingungen)*. Für Bedingungen, die sich auf mehrere Attribute Beziehen nutzen wir <a href="#14">Tabellenbedingungen</a>, dazu später. Wir können die Bedinungen noch benennen indem wir `CONSTRAINT` und dann einen Namen vor die Bedingung schreiben. Übliche Bedinungen sind:

- **Primary Key definieren:**
<br/>Hiermit geben Sie an, welches Attribut Ihre Daten eindeutig identifizert. **Achtung**: Es kann nur einen Primary Key pro Tabelle geben, mehrfachverwendung führt zu Fehlermeldungen. Wollen Sie einen PK aus mehreren Attributen definieren verwenden Sie eine <a href="#14">Tabellenbedingung</a>. <br/>*Beispiel:*
```
Kundennummer INTEGER PRIMARY KEY
```
- **Foreign Key(s) definieren:**
<br/>Hiermit geben Sie an, dass das Attribut ein Foreign Key ist und welches Attribut einer andere Tabelle referenziert wird. *Hinweis: Sie können auch Schlüsselkandidaten referenzieren, die nicht PK sind. Diese müssen dann aber mit dem UNIQUE Constraint belegt sein.*
<br/>*Beispiel:*
```
Kreditkarte TEXT FOREIGN_KEY REFERENCES Kreditkarte(Nummer)
```
- **NOT NULL:**
<br/>Weisst die Datenbank an, auf eine Eingabe in diesem Attribut zu bestehen.
<br/>*Beispiel mit Explizitem Namen:*
```
Name TEXT CONSTRAINT ImmerNachname NOT NULL
```
- **UNIQUE:** 
<br/>Das Attribut ist ein Schlüsselkandidat. Weisst die Datenbank an, auf Eideutigkeit der Werte dieses Attributs innerhalb der Tabelle zu achten. 
<br/>*Beispiel (kein gutes):*
```
Passwort TEXT UNIQUE
```
- **CHECK (Bedingung):**
<br/>Integritätsbedingung bezogen auf das aktuelle Attribut. 
<br/>*Beispiel:*
```
PLZ INTEGER CHECK ( > 9999 AND < 100000) 
```
- **DEFAULT:**
<br/>Attributwert, der genommen werden soll, wenn kein Wert angegeben wird
<br/>*Beispiel (unter Verwendung einer speziellen variablen, die den jetzigen Zeitpunkt zurückliefert):*
```
Bestelldatum TIMESTAMP DEFAULT CURRENT_TIMESTAMP
```
- <a name="serial"></a> **AUTOINCREMENT (Bedingung):**
<br/>Zähle das Attribut automatisch hoch. Hier unterscheiden sich leider die Datenbanken. Um einen Surrogate Key zu erzeugen gibt es entweder eigene Datentypen (z.B. `SERIAL`, das beschriebene Attribut, oder eigene Datenbankfunktionen [wie im Fall von SQLite](https://www.sqlite.org/autoinc.html)). Wir verwenden SQLite und lassen es also weg.
<br/>*Beispiel:*
```
Kundennummer INTEGER AUTO_INCREMENT
```

Hinweis: Ein Primary Key ist also immer `NOT NULL` und `UNIQUE`. Ob Ihre Datenbank das auch so implementiert hat müssen Sie nachlesen, oder ausprobieren. Eine zusätzliche Angabe schadet aber nicht.

Probieren wir das in unserem Beispiel aus. Vorher müssen wir aber die existierende Tabelle mit den `DROP TABLE` Befehl löschen:

In [35]:
%%sql

DROP TABLE IF EXISTS Bestellung;
DROP TABLE IF EXISTS Kunde;

CREATE TABLE Kunde(
    Kundennummer INTEGER PRIMARY KEY NOT NULL,
    Vorname TEXT,
    Nachname TEXT CONSTRAINT ImmerNachname NOT NULL,
    Wohnort TEXT,
    PLZ INTEGER CHECK (PLZ BETWEEN 9999 AND 100000),
    Adresse TEXT
);

CREATE TABLE Bestellung(
    Bestellnummer INTEGER PRIMARY KEY NOT NULL,
    Kunde INTEGER FOREIGN_KEY REFERENCES Kunde(Kundennummer),
    Bestelldatum TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

 * sqlite:///
Done.
Done.
Done.
Done.


[]

#### Übung
Passen wir doch unsere `uebung_telefonbuch` Tabelle gleich an!  Erstellen Sie die Tablle nun so, dass nur Telefonnummern aus Deutschland ( `+49....` ) eingetragen werden können. Ausserdem sollen keine doppelten Telefonnummern einfügbar sein. Implementieren Sie die Vorgabe und kommentieren Sie die fehlerhaften Reihen nach dem Test aus.

In [36]:
%%sql
-- Sie sind drann
DROP TABLE IF EXISTS uebung_telefonbuch;
CREATE TABLE uebung_telefonbuch(
    kundennummer INT PRIMARY KEY NOT NULL,
    telefonnummer TEXT UNIQUE CHECK (telefonnummer LIKE '+49%')
);

INSERT INTO uebung_telefonbuch VALUES(1, '+49071369672');
INSERT INTO uebung_telefonbuch VALUES(2, '+49022750234');
INSERT INTO uebung_telefonbuch VALUES(3, '+432038174371');
INSERT INTO uebung_telefonbuch VALUES(4, '+49959271473');
INSERT INTO uebung_telefonbuch VALUES(5, '+49022750234');

 * sqlite:///
Done.
Done.
1 rows affected.
1 rows affected.
(sqlite3.IntegrityError) CHECK constraint failed: telefonnummer LIKE '+49%'
[SQL: INSERT INTO uebung_telefonbuch VALUES(3, '+432038174371');]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


<a name="14">
</a>

## 1.4 Tabellenbedingungen

[![https://www.sqlite.org/images/syntax/table-constraint.gif](https://www.sqlite.org/images/syntax/table-constraint.gif)](https://sqlite.org/syntax/create-table-stmt.html)

Im Abschnitt vorher haben wir gesehen, wie wir einzelne Spalten mit Constraints belegen. Für Bedingungen, die sich auf mehrere Attribute beziehen bringt uns das aber nichts. Daher können wir bei der Definition einer Tabelle losgelöst von der Definition von Attributen auch noch Bedingunen für die ganze Tabelle und deren Attribute definieren. Auch diese können wir mit dem Schlüsselwort `CONSTRAINT` optional benennen. Übliche Bedinungen sind:

- **Primary Key definieren:**
<br/>Hiermit geben Sie an, welches oder welche Attribute Ihre Daten eindeutig identifizerten. <br/>*Beispiel:*
```
PRIMARY KEY(bestellnr, artikelnr)
```
- **Foreign Key(s) definieren:**
<br/>Hiermit geben Sie an, welches oder welche Attribute einen Foreign Key bilden.
<br/>*Beispiel (sehr schlechtes beispiel):*
```
FOREIGN KEY (Nachname, Vorname) REFERENCES Kunde(Nachname, Vorname)
```
- **UNIQUE:** 
<br/>Weisst die Datenbank an, auf Eideutigkeit der Werte dieses oder dieser Attribute innerhalb der Tabelle zu Achten. 
<br/>*Beispiel (kein gutes):*
```
UNIQUE(Nachname, Vorname)
```
- **CHECK (Bedingung):**
<br/>Integritätsbedingung bezogen auf portentiell alle Attribute der Tabelle. 
<br/>*Beispiel:*
```
CHECK ( (len(Nachname) + len(Vorname)) > 3) 
```


Erweitern wir also unser Schema um Artiel und Bestellungen. Wir definieren dabei PKs und FKs als Tabellenbedingung, das erhöt die Lesbarkeit.

In [37]:
%%sql

DROP TABLE IF EXISTS enthaelt;
DROP TABLE IF EXISTS ARTIKEL;

CREATE TABLE IF NOT EXISTS Artikel(
    Artikelnummer INTEGER NOT NULL,
    Name TEXT,
    Lagermenge INTEGER,
    PRIMARY KEY (Artikelnummer)
);

CREATE TABLE enthaelt(
    Bestellung INTEGER NOT NULL,
    Artikel INTEGER NOT NULL,
    Menge Integer DEFAULT 1,
    FOREIGN KEY(Artikel) REFERENCES Artikel(Artikelnummer),
    FOREIGN KEY(Bestellung) REFERENCES Bestellung(Bestellnummer),
    PRIMARY KEY(Bestellung, Artikel)
);

 * sqlite:///
Done.
Done.
Done.
Done.


[]

<a name="15">
</a>

## 1.5 Constraints testen

Bevor wir ins Testen kommen können, brauchen wir Daten. Diese legen wir nun an

In [38]:
%%sql

INSERT INTO Kunde (Vorname, Nachname, Wohnort, PLZ) VALUES('Benedikt', 'Elser', 'Deggendorf', 94469);
INSERT INTO BESTELLUNG (Kunde) VALUES(1), (1);

 * sqlite:///
1 rows affected.
2 rows affected.


[]

In [39]:
%sql SELECT * FROM KUNDE;

 * sqlite:///
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse
1,Benedikt,Elser,Deggendorf,94469,None


In [40]:
%sql SELECT * FROM Bestellung;

 * sqlite:///
Done.


Bestellnummer,Kunde,Bestelldatum
1,1,2024-01-29 23:14:48
2,1,2024-01-29 23:14:48


Testen wir nun einmal unsere Spaltenbedinungen. Wie wir oben gesehen haben wurde der `DEFAULT` Wert für den Bestellzeitpunkt bereits beachtet und unsere Bestellungen haben einen sinnvollen Wert. Wie schaut es nun bei Refrenzen auf nicht existierende Kunden aus? 

In [41]:
%sql INSERT INTO BESTELLUNG (Kunde) VALUES(12);

 * sqlite:///
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO BESTELLUNG (Kunde) VALUES(12);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


Wie erwartet ist das fehlgeschlagen. Testen wir nun unsere Constraints auf Kunden. Es folgt ein Beispiel mit falscher Postleitzahl, danach ein Beispiel ohne Nachname. Wir erwarten, dass keins der Beispiele erfolgreich ist. 

In [42]:
%sql INSERT INTO Kunde (Vorname, Nachname, Wohnort, PLZ) VALUES('Benedikt', 'Elser', 'Deggendorf', 12);

 * sqlite:///
(sqlite3.IntegrityError) CHECK constraint failed: PLZ BETWEEN 9999 AND 100000
[SQL: INSERT INTO Kunde (Vorname, Nachname, Wohnort, PLZ) VALUES('Benedikt', 'Elser' , 'Deggendorf' , 12);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [43]:
%sql INSERT INTO Kunde (Vorname, Wohnort, PLZ) VALUES('Benedikt', 'Deggendorf', 94469);

 * sqlite:///
(sqlite3.IntegrityError) NOT NULL constraint failed: Kunde.Nachname
[SQL: INSERT INTO Kunde (Vorname, Wohnort, PLZ) VALUES('Benedikt', 'Deggendorf' , 94469);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


<a name="16">
</a>

## 1.6 Views

[![https://sqlite.org/images/syntax/create-view-stmt.gif](https://sqlite.org/images/syntax/create-view-stmt.gif)](https://sqlite.org/lang_createview.html)


Ein wichtiges Feature von Relationalen Datenbanken ist die Datenunanbängigkeit. Wir hatten das 3 - Ebenen Modell gesprochen. Nun haben wir das Rüstzeug um Relationen zu definieren, es ist also an der Zeit die externe Ebene zu betrachten. Wie können wir also eine Sicht auf Tabellen erstellen, die 

- Einfacher zu handhaben sind (z.B. indem weniger Kommandos getippt, weniger Wissen über das Schema gelernt werden muss)
- Nur einen Teil der Daten zeigen (z.B. Sensible Attribute nicht anzeigt, oder ganze Reihen auslässt)
- Und trotdem keien Redundanzen erzeugt

Dieses wird in SQL als `VIEW` bezeichnet, im Folgendem implementieren wir beispielhaft eine solche. Unser Szenario ist wie folgt: Wir wollen unseren Vertriebsmitarbeitern nicht genau zeigen, wer was im Detail bestellt hat, sondern aus Datenschutzgründen lediglich eine übersicht über die Anzahl der getätigten Bestellungen pro Kunde aubsteigend sortiert geben.

*Wir verwenden im folgenden bereits Abfragen der **DML**, da eine View aus einem CREATE Teil (DDL) und dann eine Abfrage in der DML aufgebaut ist. Wir können dann die View austauschbar zum DML Select verwenden.*  In Views können unter Umständen (im Trivialfall) Daten auch geschrieben werden, dieses ist in unserer Datenbank aber nicht implementiert.

In [44]:
%%sql

DROP VIEW IF EXISTS Sales01;

CREATE VIEW Sales01
AS
SELECT Kundennummer,COUNT(*) AS AnzahlBestellungen FROM Kunde, Bestellung WHERE Bestellung.Kunde = Kunde.Kundennummer GROUP BY Kundennummer

 * sqlite:///
Done.
Done.


[]

In [45]:
%sql SELECT * FROM Sales01;

 * sqlite:///
Done.


Kundennummer,AnzahlBestellungen
1,2


Wie Sie sehen, geben wir eben keine Attribute in der Definition der View direkt an, sondern spezifizieren diese über den DML Teil. Sie können nun in der folgenden Zelle Daten in die Tabelle eintragen und sehen wie die View neue Ergebnisse zurückliefert.

In [46]:
%%sql

-- Schreiben Sie im folgenden einen oder mehrere Insert Befehle in die Kundentabelle um neue Kunden anzulegen und legen Sie dann Bestellungen für diese Kunden an.
-- Hinweis: SQL Befehle schliessen sie mit einem ; ab
-- Hinweis: Kommentare werden durch die beiden -- am Anfang der Zeile eingeleitet
-- INSERT INTO Kunde ... ;
-- INSERT INTO Bestellung ;

INSERT INTO Kunde(Nachname, PLZ) VALUES("Mustermann", 94469);
INSERT INTO Bestellung(Kunde) VALUES(2), (2), (2);

SELECT * FROM Bestellung;
SELECT * FROM Kunde;
SELECT * from Sales01;

 * sqlite:///
1 rows affected.
3 rows affected.
Done.
Done.
Done.


Kundennummer,AnzahlBestellungen
1,2
2,3


<a name="17">
</a>

## 1.7 Zugriffsrechte

Die hier verwendete Datenbank SQLite ist eine Dateibasierte Datenbank für den embedded Bereich. Sie erhalten also ein File pro Datenbank. Daher sind leider keine Zugriffsrechte implementiert und wir können das Thema nur Theoretisch betrachten.  

Gewähren und Entziehen von Zugriffsrechten
- Eigentümer kann Zugriffsrechte an Dritte vergeben 
- Diese Rechte können jederzeit widerrufen werden 
- Eigentümer behält immer alle Rechte
- Der Datenbank superuser kann aber auch den Eigentümer übergehen


#### Gewären von Rechten

Ein Nutzer ist ein username der Nutzerverwaltung des Datenbankmanagementsystems. Der Befehl baut sich wie folgt auf:
```
GRANT <Zugriffsrecht(e durch komma getrennt)> 
ON <Tabellenname oder Sichtname>
TO <Benutzername(n durch komma getrennt)
[WITH GRANT OPTION]
```

Wir können also ein oder mehrere Rechte an einer Tabelle oder Sicht an einen oder mehrere Benutzer vergeben. Optional (`WITH GRANT OPTION`) erlauben wir diesem Benutzer auch Rechte an andere weiterzugeben. Die Rechte sind:

Zugriffsrecht | Bedeutung
-----|--------
`SELECT` | Lesender Zugriff erlaub
`UPDATE` | das Ändern von Inhalten erlaubt
`UPDATE x1, x2...` | das Ändern von Inhalten der Attribute x1, ... erlaubt
`DELETE` | das Löschen von Tupeln erlaub
`INSERT` | das Einfügen neuer Tupel erlaubt
`INSERT x1, x2...` | das Einfügen neuer Tupel mit den Attributen x1, ... erlaubt
`ALL` | Alles erlaubt

Wenn wir Allen Nutzern Zugriff geben wollen, verwenden wir das Schlüsselwort `ALL` anstatt eines Benutzernamens. Verwenden Sie diesen Befehl nur mit Vorsicht. Setzen Sie Ihn eher ein beim Enzug von Nutzerrechten:

#### Enzug von Rechten

```
REVOKE 
[GRANT OPTION]
<Zugriffsrecht(e durch komma getrennt oder ALL)> 
ON <Tabellenname oder Sichtname>
FROM <Benutzername(n durch komma getrennt)
```

Man würde also typischerweise wie folgt vorgehen:

```

REVOKE ALL
ON Kunde
FROM ALL

GRANT SELECT
ON Kunde
TO sales

```

Um exklusiv dem Salesdepartment lesenden Zugriff auf Ihre Kundentabelle zu geben. Eine spannende Diskussion zu einem echten Datenbanksystem finden sie in [diesem Twitter Thread zum Thema COVID-19 App]().


<a name="18">
</a>

## 1.8 Datenbankschema

In der Vorlesung haben wir besprochen, dass eine Datenbank selbstbeschreibend sein sollte. Wie kann man das nun überprüfen? Die Antwort auf die Frage ist (wie oft bei Datenbanken) herstellerabhänig. SQLite hat eine [spezielle Tabelle](https://sqlite.org/fileformat2.html#sqlite_master), die sich `sqlite_master` nennt. Wenn wir diese Abfragen und auf die Spalte `sql`filtern finden wir alle unsere angelegten Tabellen, Sequenzen und Integritätschecks hier abgelegt:

In [47]:
%%sql
SELECT * 
FROM sqlite_master;

 * sqlite:///
Done.


type,name,tbl_name,rootpage,sql
table,Kunde,Kunde,2,"CREATE TABLE Kunde( Kundennummer INTEGER PRIMARY KEY NOT NULL, Vorname TEXT, Nachname TEXT CONSTRAINT ImmerNachname NOT NULL, Wohnort TEXT, PLZ INTEGER CHECK (PLZ BETWEEN 9999 AND 100000), Adresse TEXT)"
table,Bestellung,Bestellung,3,"CREATE TABLE Bestellung( Bestellnummer INTEGER PRIMARY KEY NOT NULL, Kunde INTEGER FOREIGN_KEY REFERENCES Kunde(Kundennummer), Bestelldatum TIMESTAMP DEFAULT CURRENT_TIMESTAMP)"
table,uebung_telefonbuch,uebung_telefonbuch,4,"CREATE TABLE uebung_telefonbuch( kundennummer INT PRIMARY KEY NOT NULL, telefonnummer TEXT UNIQUE CHECK (telefonnummer LIKE '+49%'))"
index,sqlite_autoindex_uebung_telefonbuch_1,uebung_telefonbuch,5,None
index,sqlite_autoindex_uebung_telefonbuch_2,uebung_telefonbuch,6,None
table,Artikel,Artikel,7,"CREATE TABLE Artikel( Artikelnummer INTEGER NOT NULL, Name TEXT, Lagermenge INTEGER, PRIMARY KEY (Artikelnummer))"
table,enthaelt,enthaelt,8,"CREATE TABLE enthaelt( Bestellung INTEGER NOT NULL, Artikel INTEGER NOT NULL, Menge Integer DEFAULT 1, FOREIGN KEY(Artikel) REFERENCES Artikel(Artikelnummer), FOREIGN KEY(Bestellung) REFERENCES Bestellung(Bestellnummer), PRIMARY KEY(Bestellung, Artikel))"
index,sqlite_autoindex_enthaelt_1,enthaelt,9,None
view,Sales01,Sales01,0,"CREATE VIEW Sales01ASSELECT Kundennummer,COUNT(*) AS AnzahlBestellungen FROM Kunde, Bestellung WHERE Bestellung.Kunde = Kunde.Kundennummer GROUP BY Kundennummer"


<a name="18">
</a>

## 1.9 Änderungen am Schema

Im Kapitel 1.8 haben wir gesehen, dass eine Tabellendefinition auch nur ein Eintrag in eine andere Tabelle ist. Insofern können wir nicht nur Tabellen anlegen oder löschen, sondern auch deren Schema ändern. Die Syntax ist wie folgt:

```
ALTER TABLE <Tabellenname> 
[ { ADD | MODIFY } <Attributname> Datentyp [<Spaltenbedingungen>] ]
[ DROP COLUMN  <Attributname> ]
[ { ADD | DROP } <Tabellenbedingung> ]
[ MODIFY <Default klausel> ]
[ REANME COLUMN <Attributname> TO <Attributname_neu> ]
[ REANME TO <Tabellenname_neu> ]

```

Leider implementieren auch hier verschiedene Datenbankmanagementsysteme nur teilbereiche. In SQLite ist nur folgendes implementiert:

[![](https://sqlite.org/images/syntax/alter-table-stmt.gif)](https://sqlite.org/lang_altertable.html)

So können wir also ein neues Attribut Steuernummer unseren Kunden hinzufügen:

In [48]:
%%sql

ALTER TABLE Kunde ADD COLUMN Steuernummer TEXT DEFAULT "?";
SELECT * FROM Kunde;

 * sqlite:///
Done.
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Steuernummer
1,Benedikt,Elser,Deggendorf,94469,None,?
2,None,Mustermann,None,94469,None,?


Und auch hier darf ein Verweis auf das Schichtenmodell von Datenbanken nicht fehlen. Wir haben nun an einer Relation eine Änderung durchgeführt, die externe Sicht, die wir angelegt haben bleibt davon unbehelligt.

In [49]:
%sql SELECT * FROM Sales01;

 * sqlite:///
Done.


Kundennummer,AnzahlBestellungen
1,2
2,3
